In [1]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from python_speech_features import mfcc
from scipy.io import wavfile

# Ses dosyalarından MFCC özelliklerini çıkarmak için bir fonksiyon tanımlanır.
def extract_features(file_path, mfcc_params):
    rate, audio = wavfile.read(file_path)
    # python_speech_features kütüphanesi kullanılarak MFCC özellikleri çıkarılır.
    mfcc_features = mfcc(audio, rate, **mfcc_params)
    # MFCC özelliklerinin ortalaması alınarak tek bir özellik vektörü elde edilir.
    return mfcc_features.mean(axis=0)

# Ses dosyalarının bulunduğu dizin belirlenir.
data_dir = "Sound Source 2"
categories = os.listdir(data_dir)

# MFCC parametreleri belirlenir.
mfcc_params = {
    'winlen': 0.025,   # Pencere uzunluğu (saniye cinsinden)
    'winstep': 0.01,  # Pencere adımı (saniye cinsinden)
    'numcep': 13,     # Mel-frekans cepstral katsayılarının sayısı
    'nfilt': 26,      # Filtre sayısı
    'nfft': 2048      # FFT boyutu (hızlı Fourier dönüşümü)
}

# Özelliklerin ve etiketlerin depolanacağı listeler oluşturulur.
X = []  # Özellik vektörlerini depolamak için
y = []  # Etiketleri depolamak için

# Her kategorideki ses dosyalarından özellikler çıkarılır ve X ve y listelerine eklenir.
for category in categories:
    category_dir = os.path.join(data_dir, category)
    # Dizin olup olmadığı kontrol edilir (örneğin, .DS_Store gibi gizli dosyalar atlanır).
    if not os.path.isdir(category_dir):
        continue
    for file_name in os.listdir(category_dir):
        if file_name.startswith('.'):
            continue
        file_path = os.path.join(category_dir, file_name)
        features = extract_features(file_path, mfcc_params)
        X.append(features)
        y.append(category)

# Listeler numpy dizilerine dönüştürülür.
X = np.array(X)
y = np.array(y)

# Veri kümesi eğitim, doğrulama ve test setlerine ayrılır.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Rastgele Orman modelinin hiperparametrelerini aramak için Grid Search yapılır.
n_estimators_values = [50, 100, 150]
max_depth_values = [None, 10, 20, 30]
min_samples_split_values = [2, 5, 10]
min_samples_leaf_values = [1, 2, 4]

best_score = 0.0
best_n_estimators = 0
best_max_depth = 0
best_min_samples_split = 0
best_min_samples_leaf = 0

# Grid Search ile en iyi performansı gösteren hiperparametreler bulunur.
for n_estimators in n_estimators_values:
    for max_depth in max_depth_values:
        for min_samples_split in min_samples_split_values:
            for min_samples_leaf in min_samples_leaf_values:
                forest = RandomForestClassifier(n_estimators=n_estimators,
                                                max_depth=max_depth,
                                                min_samples_split=min_samples_split,
                                                min_samples_leaf=min_samples_leaf,
                                                random_state=42)
                forest.fit(X_train, y_train)
                score = forest.score(X_val, y_val)
                # Doğrulama setinde elde edilen en iyi performans kaydedilir.
                if score > best_score:
                    best_score = score
                    best_n_estimators = n_estimators
                    best_max_depth = max_depth
                    best_min_samples_split = min_samples_split
                    best_min_samples_leaf = min_samples_leaf

# En iyi performansı gösteren hiperparametrelerle model tekrar eğitilir.
forest = RandomForestClassifier(n_estimators=best_n_estimators,
                                max_depth=best_max_depth,
                                min_samples_split=best_min_samples_split,
                                min_samples_leaf=best_min_samples_leaf,
                                random_state=42)
forest.fit(X_train, y_train)

# En iyi modelin test seti üzerindeki performansı hesaplanır.
test_score = forest.score(X_test, y_test)

# Sonuçlar
print("Best Validation Score: {:.2f}".format(best_score))
print("Best Test Score: {:.2f}".format(test_score))
print("Best n_estimators (ağaç sayısı)", best_n_estimators)
print("Best max_depth(maksimum derinlik)", best_max_depth)
print("Best min_samples_split(dallanma için minimum örnek sayısı)", best_min_samples_split)
print("Best min_samples_leaf  (yaprak düğümü için minimum örnek sayısı)", best_min_samples_leaf)


Best Validation Score: 0.81
Best Test Score: 0.73
Best n_estimators (ağaç sayısı) 100
Best max_depth(maksimum derinlik) None
Best min_samples_split(dallanma için minimum örnek sayısı) 2
Best min_samples_leaf  (yaprak düğümü için minimum örnek sayısı) 2
